In [ ]:
import pandas as pd

teste_df = pd.read_excel("teste.xlsx")
cor_df = pd.read_excel("cor.xlsx")
categorias_df = pd.read_excel("categorias.xlsx")
marcas_df = pd.read_excel("marcas.xlsx")
fornecedores_df = pd.read_excel("fornecedores.xlsx")

teste_df = teste_df.rename(columns={'REF KINGS' : 'Referências', 'DESCRIÇÃO' : 'Nome do Produto', 'ATACADO' : 'Valor atacado', 'VAR' : 'Valor varejo', 'LINHA' : 'Coleção', 'COD. BARRAS': 'Código de Barras'})

teste_df ['Status do Produto'] = 1
teste_df ['SKU'] = teste_df ['Referências']
teste_df ['Exibir produto na home'] = '0'
teste_df ['Peso do Produto'] = ''
teste_df ['Tamanho do Produto'] = ''
teste_df ['Área de Disponibilidade'] = '{P}'
teste_df ['Descrição do Produto'] = ''
teste_df ['Grade ID'] = 21

teste_df['Referências'] = teste_df['Referências'].str.slice(0, -4)

teste_df = teste_df.merge(cor_df, on= 'COR').rename(columns={'ID' : 'Cor'})
teste_df = teste_df.merge(fornecedores_df, on= 'FORNECEDOR').rename(columns={'ID' : 'Fornecedor ID'})
teste_df = teste_df.merge(categorias_df, on= 'CATEGORIA').rename(columns={'ID' : 'Categoria ID'})
teste_df = teste_df.merge(marcas_df, on= 'MARCA').rename(columns={'ID' : 'Marca ID'})

teste_df['Marca ID'] = teste_df['Marca ID'].astype(int)
teste_df['Fornecedor ID'] = teste_df['Fornecedor ID'].astype(int)


teste_df = teste_df.drop_duplicates(subset=['SKU'])
teste_df = teste_df [['Referências' , 'SKU', 'Status do Produto', 'Nome do Produto', 'Categoria ID', 'Grade ID', 'Exibir produto na home', 'Valor atacado', 'Peso do Produto', 'Tamanho do Produto', 'Marca ID', 'Fornecedor ID', 'Código de Barras', 'Coleção', 'Valor varejo', 'Área de Disponibilidade', 'Descrição do Produto', 'Cor']]

teste_df.dtypes

print(teste_df.to_markdown())

teste_df.to_excel('upload.xlsx', index=False)


|    | Referências   | SKU              |   Status do Produto | Nome do Produto                                              |   Categoria ID |   Grade ID |   Exibir produto na home |   Valor atacado | Peso do Produto   | Tamanho do Produto   |   Marca ID |   Fornecedor ID |   Código de Barras | Coleção                      |   Valor varejo | Área de Disponibilidade   | Descrição do Produto   |   Cor |
|---:|:--------------|:-----------------|--------------------:|:-------------------------------------------------------------|---------------:|-----------:|-------------------------:|----------------:|:------------------|:---------------------|-----------:|----------------:|-------------------:|:-----------------------------|---------------:|:--------------------------|:-----------------------|------:|
|  0 | CAM1VER26TOY  | CAM1VER26TOY-001 |                   1 | Camiseta oversized Kings Collab pixar Toy Story crew         |              3 |         21 |                        0 |     